# Dandelion for TCR gamma/delta reannotation

![dandelion_logo](img/dandelion_logo_illustration.png)

In Cell Ranger 3.1.0, the VDJ algorithm was changed to favour TCR alpha/beta annotation. Since then, calling gamma/delta chains has become challenging, and 10X support recommends using Cell Ranger 3.0.2 when working with gamma/delta-rich libraries. 

However, the contigs themselves are still accurately reconstructed, just not annotated correctly. It may be desirable to use a newer Cell Ranger version for access to some previously unavailable run options, like specifying custom enrichment primers. In those cases, the contigs can be reannotated via `dandelion` to yield functional output.

Prepare your input like you [normally would](https://sc-dandelion.readthedocs.io/en/latest/notebooks/singularity_preprocessing.html) for preprocessing - one folder per sample, holding the Cell Ranger V(D)J output. Then for each `$SAMPLE`, do the following:

```bash
cut -f3-4 -d, $SAMPLE/all_contig_annotations.csv | grep True | cut -f1 -d, > $SAMPLE/hiconf.txt

head -n 1 $SAMPLE/all_contig_annotations.csv > $SAMPLE/hiconf_contig_annotations.csv

grep -f $SAMPLE/hiconf.txt $SAMPLE/all_contig_annotations.csv >> $SAMPLE/hiconf_contig_annotations.csv 

grep -A1 --no-group-separator -f $SAMPLE/hiconf.txt $SAMPLE/all_contig.fasta > $SAMPLE/hiconf_contig.fasta

rm $SAMPLE/hiconf.txt
```

Cellular assignment of contigs tends to be wonky for gamma/delta-rich libraries as well, so this goes back to the unfiltered contig file and identifies the names of all contigs Cell Ranger deems high confidence (this part works fine). 

This contig list is then used to fish out the contig sequence and annotation into a new pair of Dandelion compliant input files. Once done, the holder file with the high confidence contig names is removed.

Once all your samples are processed this way, you can run `dandelion`:

```bash
singularity run -B $PWD ~/sc-dandelion_latest.sif dandelion-preprocess --chain TR \
        --file_prefix hiconf \
        --keep_trailing_hyphen_number
```

For each `$SAMPLE`, the reannotated high confidence contigs can be found at `$SAMPLE/dandelion/hiconf_contig_igblast_db-pass.tsv`.